In [1]:
#openai LLM API와 통신할 때 쓰는 요소들
#prompt
#models
#output parsers

In [3]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.8 MB/s eta 0:00:00


In [4]:
import os
from google.colab import userdata

#사용할 openai api key를 등록
os.environ['OPENAI_API_KEY'] = userdata.get('AI_KEY')

In [5]:
#업데이트된 무료버전 chatGPT
llm_model = 'gpt-3.5-turbo-0301'

In [7]:
import openai

In [8]:
def get_completion(prompt, model=llm_model):
  #systemprompt가 없이, 내가 묻고자 하는 것만 전달하는 message
  message = [{"role":"user",
              "content" : prompt}]

  response = openai.ChatCompletion.create(
      model = model,
      messages = message,
      temperature = 0.1
  )

  return response.choices[0].message['content']

In [9]:
result = get_completion("What is the 2 + 2 ?")

In [10]:
print(result)

As an AI language model, I can say that the answer to 2 + 2 is 4.


## 모델에게 특정한 행동 양식 지정하기 -> 프롬프트

In [11]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [12]:
style = """American English \
in a calm and respectful tone
"""

In [13]:
prompt = f"Translate {customer_email} to {style}"

print(prompt)

Translate 
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
 to American English in a calm and respectful tone



In [15]:
customer_email = "뺘뀌별레 냐와 꺠꼬쌩"

In [16]:
style = "Korean, calm and respectful"

In [18]:
prompt = f"Translate {customer_email} to {style}"

print(prompt)

Translate 뺘뀌별레 냐와 꺠꼬쌩 to Korean, calm and respectful


In [19]:
result = get_completion(prompt)
print(result)

I'm sorry, but "뺘뀌별레 냐와 꺠꼬쌩" is not a phrase or sentence in any language that I am aware of. It appears to be a nonsensical combination of letters and sounds. Can you provide more context or information so I can assist you better?


In [ ]:
#"I am quite upset that the lid of my blender came off and caused my kitchen walls \n
#to be covered in smoothie. Additionally, the warranty does not cover the cost of \n
#cleaning up my kitchen. I would greatly appreciate your assistance at this time, my friend."

## 같은 작업(프롬프트)을 랭체인을 통해 실행하기!

In [20]:
!pip install langchain==0.1.0

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtim

In [21]:
#openai llm에게 "대화체로 된 " 답변을 요구할 때 쓰는 모듈
from langchain.chat_models import ChatOpenAI

In [22]:
chat = ChatOpenAI(temperature=0.1, model=llm_model)

In [28]:
prompt = f"Translate {customer_email} to {style}"

print(prompt)

Translate 뺘뀌별레 냐와 꺠꼬쌩 to Korean, calm and respectful


In [23]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(prompt)

In [29]:
customer_message = prompt_template.format_messages(
    style = customer_email,
    text = style
)

In [30]:
response = chat(customer_message)

In [31]:
response

AIMessage(content='I\'m sorry, but "뺘뀌별레 냐와 꺠꼬쌩" does not make sense in any language. Could you please provide the correct phrase or sentence that you want me to translate?')

## Output Parser -> 내가 llm에게 "특정한 양식"을 갖춘 답변을 요구할 때

In [32]:
#가상의 구매자 리뷰
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

In [33]:
#위의 리뷰를 어떤 형식으로 돌려받고 싶은지
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [35]:
from langchain.prompts import ChatPromptTemplate

#Chat에 대해, 특정한 양식을 요구하는 review_template을 정의해줌
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [36]:
#prompt_template에게 "review_template"이라는 정해진 양식으로 결과를 도출할 것을 요구
#재료가 될 리뷰는 text = customer_review
message = prompt_template.format_messages(text=customer_review)

In [37]:
chat = ChatOpenAI(temperature=0.1, model=llm_model)
response = chat(message)
print(response)

content='{\n    "gift": true,\n    "delivery_days": 2,\n    "price_value": ["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]\n}'


In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""